In [1]:
import Pkg
Pkg.add("JuMP")
using JuMP
using Gurobi
using CSV

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


    Fetching: [

    Fetching: [===============>                         ]  

    Fetching: [

    Fetching: [==============================>          

    Fetching: [

Fetching: [========================================>]  100.0 %

  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]


## Pre-processing: Loading Data

#### a) Interstate distances (in miles)

In [2]:
distances = CSV.read("Distance of Transportation Matrix.csv")

# 45cents per pound
# 11 pounds per ventilator
# each ventilator costs $4.95 per mile

,Column1,AL,AZ,AR,CA,CO,CT,DE,FL,GA,ID,IL
,String,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,AL,0,1596,459,2164,1458,1160,931,547,274,2146,731
2,AZ,1596,0,1235,735,645,2435,2320,2149,1884,988,1544
3,AR,459,1235,0,1803,1047,1308,1087,993,650,1767,471
4,CA,2164,735,1803,0,1140,2991,2847,2705,2452,908,2083
5,CO,1458,645,1047,1140,0,1919,1832,1965,1623,867,1011
6,CT,1160,2435,1308,2991,1919,0,284,1243,1014,2526,994
7,DE,931,2320,1087,2847,1832,284,0,1013,783,2441,861
8,FL,547,2149,993,2705,1965,1243,1013,0,366,2648,1185
9,GA,274,1884,650,2452,1623,1014,783,366,0,2306,843


#### b) Demand of each state for 8 weeks (Week of Mar 2-Apr 20)

In [3]:
dem = CSV.read("test.csv")

,Column1,week 1,week 2,week 3,week 4,week 5,week 6,week 7,week 8
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,AL,0.0,30.0,240.0,940.0,2200.0,3880.0,5210.0,6539.0
2,AR,0.0,14.0,154.0,504.0,854.0,1344.0,2114.0,3017.0
3,AZ,0.0,15.0,365.0,1415.0,2815.0,3865.0,5195.0,6716.0
4,CA,20.0,90.0,4990.0,11290.0,17590.0,25990.0,34740.0,44997.0
5,CO,10.0,40.0,880.0,3610.0,5710.0,7810.0,10260.0,13441.0
6,CT,0.0,20.0,720.0,3520.0,7720.0,13320.0,19620.0,25269.0
7,DE,0.0,14.0,84.0,294.0,924.0,1764.0,2814.0,4162.0
8,FL,0.0,50.0,1450.0,4950.0,10550.0,17550.0,23850.0,32130.0
9,GA,0.0,20.0,1420.0,4220.0,8770.0,14370.0,18570.0,22746.0


## Initializing Constant Data (Unchanging throughout 8 weeks)

In [4]:
states = [:AL, :AR, :AZ, :CA, :CO, :CT, :DE, :FL, :GA, :IA,
:ID, :IL, :IN, :KS, :KY, :LA, :MA, :MD, :ME, :MI, :MN, :MO, :MS, 
:MT, :NC, :ND, :NE, :NH, :NJ, :NM, :NV, :NY, :OH, :OK, :OR, 
:PA, :RI, :SC, :SD, :TN, :TX, :UT, :VA, :VT, :WA, :WI, :WV, :WY]

## FACTORY/PRODUCTION DATA
fac = [:CA, :MS, :NE, :NJ, :OH]
# Factory Capacity
cap = Dict()
cap[:CA] = 1000
cap[:MS] = 500
cap[:NE] = 500
cap[:NJ] = 2000
cap[:OH] = 900

# Cost of Production for each Factory
prod = Dict()
prod[:CA] = 1000 # ETC
prod[:MS] = 1000
prod[:NE] = 1000
prod[:NJ] = 1000
prod[:OH] = 1000

## TRANSPORTATION DATA
# Cost of transporting one ventilator from one state to another
transport = Dict()
i = 0
for origin in states
    i += 1
    j = 1
    for dest in states
        j += 1
        transport[(origin, dest)] = round(distances[i, j] * 4.95, digits=2)
    end
end

## WEEK 1 MODEL

### Demand & Existing Number of Ventilators Data

In [5]:
## DEMAND
# Projected demand for state i next week
demand = Dict()
i = 0
for s in states
    i += 1
    demand[s] = ceil(dem[i, 2]*0.08) #assume 8% of cases need ventilators
end

# Existing # of ventilators from previous week
# Initialize to 0 for Week 1
num_venti = Dict()
for s in states
    num_venti[s] = 0
end

### Variables, Objective, Constraints

In [10]:
nrm = Model(Gurobi.Optimizer)

# How many ventilators to send from one state to another
@variable(nrm, x[o=states, d=states] >= 0)

# How many ventilators to produce from each factory
@variable(nrm, 0 <= y[f=fac] <= cap[f])

## OBJECTIVE FUNCTION
@objective(nrm, Min,
        sum(transport[(o, d)]*x[o, d] for o in states, d in states if o != d) + 
        sum(prod[f]*y[f] for f in fac))

## CONSTRAINTS
for s in states
    # if state produces ventilators, add y[state]
    if s in fac
        @constraint(nrm, num_venti[s] + sum(x[o, s] for o in states if o != s) - 
        sum(x[s, d] for d in states if d != s) + y[s] >= demand[s] * 0.5) # all states must have at least 50% demand met
    else
        @constraint(nrm, num_venti[s] + sum(x[o, s] for o in states if o != s) - 
        sum(x[s, d] for d in states if d != s) >= demand[s] *0.5)
    end
end

Academic license - for non-commercial use only


### SOLVE

In [11]:
optimize!(nrm)

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 48 rows, 2309 columns and 4517 nonzeros
Model fingerprint: 0x98fd1130
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+02, 2e+04]
  Bounds range     [5e+02, 2e+03]
  RHS range        [5e-01, 2e+00]
Presolve removed 0 rows and 48 columns
Presolve time: 0.00s
Presolved: 48 rows, 2261 columns, 4517 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.687500e+00   0.000000e+00      0s
      42    2.2760700e+04   0.000000e+00   0.000000e+00      0s

Solved in 42 iterations and 0.00 seconds
Optimal objective  2.276070000e+04


In [12]:
value.(y)

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:CA, :MS, :NE, :NJ, :OH]
And data, a 5-element Array{Float64,1}:
 3.0
 1.0
 1.5
 0.5
 0.0

In [9]:
objective_value(nrm)

22760.7

## WEEK 2

### Updating Number of Ventilators and Projected Demand

In [166]:
for s in states
    # if state produces ventilators, add y[state]
    if s in fac
        num_venti[s] += ceil(sum(value.(x[o, s]) for o in states if o != s)) - 
        ceil(sum(value.(x[s, d]) for d in states if d != s)) + ceil(value.(y[s]))
    else
        num_venti[s] += ceil(sum(value.(x[o, s]) for o in states if o != s)) - 
        ceil(sum(value.(x[s, d]) for d in states if d != s))
    end
end

## DEMAND
# Projected demand for state i next week
demand = Dict()
i = 0
for s in states
    i += 1
    demand[s] = dem[i, 3]*0.08
end

### Running Model

In [172]:
## DECISION VARIABLES
nrm = Model(Gurobi.Optimizer)

# How many ventilators to send from one state to another
@variable(nrm, x[o=states, d=states] >= 0)

# How many ventilators to produce from each factory
@variable(nrm, 0 <= y[f=fac] <= cap[f])

## OBJECTIVE FUNCTION
@objective(nrm, Min,
        sum(transport[(o, d)]*x[o, d] for o in states, d in states if o != d) + 
        sum(prod[f]*y[f] for f in fac))

## CONSTRAINTS
for s in states
    # if state produces ventilators, add y[state]
    if s in fac
        @constraint(nrm, num_venti[s] + sum(x[o, s] for o in states if o != s) - 
        sum(x[s, d] for d in states if d != s) + y[s] >= demand[s] * 0.5)
    else
        @constraint(nrm, num_venti[s] + sum(x[o, s] for o in states if o != s) - 
        sum(x[s, d] for d in states if d != s) >= demand[s] *0.5)
    end
end

## SOLVE ##
optimize!(nrm)
value.(y)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 48 rows, 2309 columns and 4517 nonzeros
Model fingerprint: 0xc9c769b6
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 2e+04]
  Bounds range     [5e+02, 2e+03]
  RHS range        [4e-02, 8e+02]
Presolve removed 0 rows and 48 columns
Presolve time: 0.00s
Presolved: 48 rows, 2261 columns, 4517 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   8.504100e+02   0.000000e+00      0s
     104    4.4752772e+06   0.000000e+00   0.000000e+00      0s

Solved in 104 iterations and 0.00 seconds
Optimal objective  4.475277218e+06


1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:CA, :MS, :NE, :NJ, :OH]
And data, a 5-element Array{Float64,1}:
  25.840000000000003
 500.0
   7.36
 335.96000000000015
  13.76

## WEEK 3

### Updating Number of Ventilators and Projected Demand

In [173]:
for s in states
    # if state produces ventilators, add y[state]
    if s in fac
        num_venti[s] += ceil(sum(value.(x[o, s]) for o in states if o != s)) - 
        ceil(sum(value.(x[s, d]) for d in states if d != s)) + ceil(value.(y[s]))
    else
        num_venti[s] += ceil(sum(value.(x[o, s]) for o in states if o != s)) - 
        ceil(sum(value.(x[s, d]) for d in states if d != s))
    end
end

## DEMAND
# Projected demand for state i next week
demand = Dict()
i = 0
for s in states
    i += 1
    demand[s] = dem[i, 4]*0.08
end

### Running Model

In [175]:
## DECISION VARIABLES
nrm = Model(Gurobi.Optimizer)

# How many ventilators to send from one state to another
@variable(nrm, x[o=states, d=states] >= 0)

# How many ventilators to produce from each factory
@variable(nrm, 0 <= y[f=fac] <= cap[f])

## OBJECTIVE FUNCTION
@objective(nrm, Min,
        sum(transport[(o, d)]*x[o, d] for o in states, d in states if o != d) + 
        sum(prod[f]*y[f] for f in fac))

## CONSTRAINTS
for s in states
    # if state produces ventilators, add y[state]
    if s in fac
        @constraint(nrm, num_venti[s] + sum(x[o, s] for o in states if o != s) - 
        sum(x[s, d] for d in states if d != s) + y[s] >= demand[s] * 0.5)
    else
        @constraint(nrm, num_venti[s] + sum(x[o, s] for o in states if o != s) - 
        sum(x[s, d] for d in states if d != s) >= demand[s] *0.5)
    end
end

## SOLVE ##
optimize!(nrm)
value.(y)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 48 rows, 2309 columns and 4517 nonzeros
Model fingerprint: 0x9df9a5f0
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 2e+04]
  Bounds range     [5e+02, 2e+03]
  RHS range        [2e-01, 2e+03]
Presolve removed 0 rows and 48 columns
Presolve time: 0.00s
Presolved: 48 rows, 2261 columns, 4517 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.748865e+03   0.000000e+00      0s
     108    1.2452864e+07   0.000000e+00   0.000000e+00      0s

Solved in 108 iterations and 0.00 seconds
Optimal objective  1.245286421e+07


1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:CA, :MS, :NE, :NJ, :OH]
And data, a 5-element Array{Float64,1}:
  275.92
  500.0
  185.56
 1500.0
  618.7200000000005

## WEEK 4

### Updating Number of Ventilators and Projected Demand

In [176]:
for s in states
    # if state produces ventilators, add y[state]
    if s in fac
        num_venti[s] += ceil(sum(value.(x[o, s]) for o in states if o != s)) - 
        ceil(sum(value.(x[s, d]) for d in states if d != s)) + ceil(value.(y[s]))
    else
        num_venti[s] += ceil(sum(value.(x[o, s]) for o in states if o != s)) - 
        ceil(sum(value.(x[s, d]) for d in states if d != s))
    end
end

## DEMAND
# Projected demand for state i next week
demand = Dict()
i = 0
for s in states
    i += 1
    demand[s] = dem[i, 5]*0.08
end

### Running Model

In [177]:
## DECISION VARIABLES
nrm = Model(Gurobi.Optimizer)

# How many ventilators to send from one state to another
@variable(nrm, x[o=states, d=states] >= 0)

# How many ventilators to produce from each factory
@variable(nrm, 0 <= y[f=fac] <= cap[f])

## OBJECTIVE FUNCTION
@objective(nrm, Min,
        sum(transport[(o, d)]*x[o, d] for o in states, d in states if o != d) + 
        sum(prod[f]*y[f] for f in fac))

## CONSTRAINTS
for s in states
    # if state produces ventilators, add y[state]
    if s in fac
        @constraint(nrm, num_venti[s] + sum(x[o, s] for o in states if o != s) - 
        sum(x[s, d] for d in states if d != s) + y[s] >= demand[s] * 0.5)
    else
        @constraint(nrm, num_venti[s] + sum(x[o, s] for o in states if o != s) - 
        sum(x[s, d] for d in states if d != s) >= demand[s] *0.5)
    end
end

## SOLVE ##
optimize!(nrm)
value.(y)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 48 rows, 2309 columns and 4517 nonzeros
Model fingerprint: 0xd412a781
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 2e+04]
  Bounds range     [5e+02, 2e+03]
  RHS range        [1e+00, 2e+03]
Presolve removed 0 rows and 48 columns
Presolve time: 0.00s
Presolved: 48 rows, 2261 columns, 4517 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.661520e+03   0.000000e+00      0s

Solved in 166 iterations and 0.01 seconds
Infeasible model


1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:CA, :MS, :NE, :NJ, :OH]
And data, a 5-element Array{Float64,1}:
 1954.5200000000004
  500.0
  500.0
 1500.0
  900.0

## WEEK 5
##### After this point, the fairness constraint needs to be lowered

In [178]:
for s in states
    # if state produces ventilators, add y[state]
    if s in fac
        num_venti[s] += ceil(sum(value.(x[o, s]) for o in states if o != s)) - 
        ceil(sum(value.(x[s, d]) for d in states if d != s)) + ceil(value.(y[s]))
    else
        num_venti[s] += ceil(sum(value.(x[o, s]) for o in states if o != s)) - 
        ceil(sum(value.(x[s, d]) for d in states if d != s))
    end
end

## DEMAND
# Projected demand for state i next week
demand = Dict()
i = 0
for s in states
    i += 1
    demand[s] = dem[i, 6]*0.08
end

In [181]:
## DECISION VARIABLES
nrm = Model(Gurobi.Optimizer)

# How many ventilators to send from one state to another
@variable(nrm, x[o=states, d=states] >= 0)

# How many ventilators to produce from each factory
@variable(nrm, 0 <= y[f=fac] <= cap[f])

## OBJECTIVE FUNCTION
@objective(nrm, Min,
        sum(transport[(o, d)]*x[o, d] for o in states, d in states if o != d) + 
        sum(prod[f]*y[f] for f in fac))

## CONSTRAINTS
for s in states
    # if state produces ventilators, add y[state]
    if s in fac
        @constraint(nrm, num_venti[s] + sum(x[o, s] for o in states if o != s) - 
        sum(x[s, d] for d in states if d != s) + y[s] >= demand[s] * 0.4)
    else
        @constraint(nrm, num_venti[s] + sum(x[o, s] for o in states if o != s) - 
        sum(x[s, d] for d in states if d != s) >= demand[s] *0.4)
    end
end

## SOLVE ##
optimize!(nrm)
value.(y)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 48 rows, 2309 columns and 4517 nonzeros
Model fingerprint: 0x1a60c5b7
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 2e+04]
  Bounds range     [5e+02, 2e+03]
  RHS range        [2e+00, 7e+02]
Presolve removed 0 rows and 48 columns
Presolve time: 0.00s
Presolved: 48 rows, 2261 columns, 4517 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.685894e+03   0.000000e+00      0s
     111    7.7536392e+06   0.000000e+00   0.000000e+00      0s

Solved in 111 iterations and 0.00 seconds
Optimal objective  7.753639217e+06


1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:CA, :MS, :NE, :NJ, :OH]
And data, a 5-element Array{Float64,1}:
  201.66400000000004
  500.0
  500.0
 1318.712000000001
  900.0

## WEEK 6

In [182]:
for s in states
    # if state produces ventilators, add y[state]
    if s in fac
        num_venti[s] += ceil(sum(value.(x[o, s]) for o in states if o != s)) - 
        ceil(sum(value.(x[s, d]) for d in states if d != s)) + ceil(value.(y[s]))
    else
        num_venti[s] += ceil(sum(value.(x[o, s]) for o in states if o != s)) - 
        ceil(sum(value.(x[s, d]) for d in states if d != s))
    end
end

## DEMAND
# Projected demand for state i next week
demand = Dict()
i = 0
for s in states
    i += 1
    demand[s] = dem[i, 7]*0.08
end

In [186]:
## DECISION VARIABLES
nrm = Model(Gurobi.Optimizer)

# How many ventilators to send from one state to another
@variable(nrm, x[o=states, d=states] >= 0)

# How many ventilators to produce from each factory
@variable(nrm, 0 <= y[f=fac] <= cap[f])

## OBJECTIVE FUNCTION
@objective(nrm, Min,
        sum(transport[(o, d)]*x[o, d] for o in states, d in states if o != d) + 
        sum(prod[f]*y[f] for f in fac))

## CONSTRAINTS
for s in states
    # if state produces ventilators, add y[state]
    if s in fac
        @constraint(nrm, num_venti[s] + sum(x[o, s] for o in states if o != s) - 
        sum(x[s, d] for d in states if d != s) + y[s] >= demand[s] * 0.35)
    else
        @constraint(nrm, num_venti[s] + sum(x[o, s] for o in states if o != s) - 
        sum(x[s, d] for d in states if d != s) >= demand[s] *0.35)
    end
end

## SOLVE ##
optimize!(nrm)
value.(y)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 48 rows, 2309 columns and 4517 nonzeros
Model fingerprint: 0x8a84431e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 2e+04]
  Bounds range     [5e+02, 2e+03]
  RHS range        [1e+00, 1e+03]
Presolve removed 0 rows and 48 columns
Presolve time: 0.00s
Presolved: 48 rows, 2261 columns, 4517 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.323727e+03   0.000000e+00      0s
     118    1.1908314e+07   0.000000e+00   0.000000e+00      0s

Solved in 118 iterations and 0.00 seconds
Optimal objective  1.190831429e+07


1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:CA, :MS, :NE, :NJ, :OH]
And data, a 5-element Array{Float64,1}:
  492.196
  500.0
  500.0
 1500.0
  900.0

## WEEK 7

In [187]:
for s in states
    # if state produces ventilators, add y[state]
    if s in fac
        num_venti[s] += ceil(sum(value.(x[o, s]) for o in states if o != s)) - 
        ceil(sum(value.(x[s, d]) for d in states if d != s)) + ceil(value.(y[s]))
    else
        num_venti[s] += ceil(sum(value.(x[o, s]) for o in states if o != s)) - 
        ceil(sum(value.(x[s, d]) for d in states if d != s))
    end
end

## DEMAND
# Projected demand for state i next week
demand = Dict()
i = 0
for s in states
    i += 1
    demand[s] = dem[i, 8]*0.08
end

In [188]:
## DECISION VARIABLES
nrm = Model(Gurobi.Optimizer)

# How many ventilators to send from one state to another
@variable(nrm, x[o=states, d=states] >= 0)

# How many ventilators to produce from each factory
@variable(nrm, 0 <= y[f=fac] <= cap[f])

## OBJECTIVE FUNCTION
@objective(nrm, Min,
        sum(transport[(o, d)]*x[o, d] for o in states, d in states if o != d) + 
        sum(prod[f]*y[f] for f in fac))

## CONSTRAINTS
for s in states
    # if state produces ventilators, add y[state]
    if s in fac
        @constraint(nrm, num_venti[s] + sum(x[o, s] for o in states if o != s) - 
        sum(x[s, d] for d in states if d != s) + y[s] >= demand[s] * 0.35)
    else
        @constraint(nrm, num_venti[s] + sum(x[o, s] for o in states if o != s) - 
        sum(x[s, d] for d in states if d != s) >= demand[s] *0.35)
    end
end

## SOLVE ##
optimize!(nrm)
value.(y)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 48 rows, 2309 columns and 4517 nonzeros
Model fingerprint: 0x2b61de2f
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 2e+04]
  Bounds range     [5e+02, 2e+03]
  RHS range        [2e+00, 5e+02]
Presolve removed 0 rows and 48 columns
Presolve time: 0.00s
Presolved: 48 rows, 2261 columns, 4517 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.469802e+03   0.000000e+00      0s
     140    1.3977994e+07   0.000000e+00   0.000000e+00      0s

Solved in 140 iterations and 0.00 seconds
Optimal objective  1.397799352e+07


1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:CA, :MS, :NE, :NJ, :OH]
And data, a 5-element Array{Float64,1}:
  632.0880000000001
  500.0
  500.0
 1500.0
  900.0

## WEEK 8

In [191]:
for s in states
    # if state produces ventilators, add y[state]
    if s in fac
        num_venti[s] += ceil(sum(value.(x[o, s]) for o in states if o != s)) - 
        ceil(sum(value.(x[s, d]) for d in states if d != s)) + ceil(value.(y[s]))
    else
        num_venti[s] += ceil(sum(value.(x[o, s]) for o in states if o != s)) - 
        ceil(sum(value.(x[s, d]) for d in states if d != s))
    end
end

## DEMAND
# Projected demand for state i next week
demand = Dict()
i = 0
for s in states
    i += 1
    demand[s] = dem[i, 9]*0.08
end

In [195]:
## DECISION VARIABLES
nrm = Model(Gurobi.Optimizer)

# How many ventilators to send from one state to another
@variable(nrm, x[o=states, d=states] >= 0)

# How many ventilators to produce from each factory
@variable(nrm, 0 <= y[f=fac] <= cap[f])

## OBJECTIVE FUNCTION
@objective(nrm, Min,
        sum(transport[(o, d)]*x[o, d] for o in states, d in states if o != d) + 
        sum(prod[f]*y[f] for f in fac))

## CONSTRAINTS
for s in states
    # if state produces ventilators, add y[state]
    if s in fac
        @constraint(nrm, num_venti[s] + sum(x[o, s] for o in states if o != s) - 
        sum(x[s, d] for d in states if d != s) + y[s] >= demand[s] * 0.32)
    else
        @constraint(nrm, num_venti[s] + sum(x[o, s] for o in states if o != s) - 
        sum(x[s, d] for d in states if d != s) >= demand[s] *0.32)
    end
end

## SOLVE ##
optimize!(nrm)
value.(y)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 48 rows, 2309 columns and 4517 nonzeros
Model fingerprint: 0x5440063d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 2e+04]
  Bounds range     [5e+02, 2e+03]
  RHS range        [5e-01, 9e+02]
Presolve removed 0 rows and 48 columns
Presolve time: 0.00s
Presolved: 48 rows, 2261 columns, 4517 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.210934e+03   0.000000e+00      0s
     118    1.2229374e+07   0.000000e+00   0.000000e+00      0s

Solved in 118 iterations and 0.01 seconds
Optimal objective  1.222937384e+07


1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:CA, :MS, :NE, :NJ, :OH]
And data, a 5-element Array{Float64,1}:
  884.6528000000017
  500.0
  500.0
 1500.0
  900.0